# Store Sales Time Series Forecasting - Debug Notebook

This notebook debugs and fixes the issue where all predictions are coming out as $0.00 in our time series forecasting model for Favorita store sales.

## Problem Analysis
- Model is training successfully (RMSLE: 0.55333)
- Predictions shape is correct (16, 1782)
- But all final predictions are $0.00

## Goals
1. Debug the prediction-to-submission pipeline
2. Fix the data merging issues
3. Create proper submission format
4. Validate the final predictions

## 1. Data Loading and Preparation

Load the necessary libraries and data files for debugging the prediction pipeline.

In [ ]:
import pandas as pd
import numpy as np
import joblib
import warnings
warnings.filterwarnings('ignore')

print("Libraries loaded successfully!")

# Load the saved model if it exists
try:
    model = joblib.load('time_series_model.pkl')
    print("✓ Saved model loaded successfully")
except FileNotFoundError:
    print("❌ No saved model found. Need to run main.py first.")
    model = None

Libraries loaded successfully!
✓ Saved model loaded successfully


In [13]:
# Load the problematic submission file
try:
    submission_zero = pd.read_csv('submission_notebook_method.csv')
    print(f"✓ Submission file loaded: {len(submission_zero)} rows")
    print("Sales statistics:")
    print(f"  Min: ${submission_zero['sales'].min():.2f}")
    print(f"  Max: ${submission_zero['sales'].max():.2f}")
    print(f"  Mean: ${submission_zero['sales'].mean():.2f}")
    print(f"  Unique values: {submission_zero['sales'].nunique()}")
    
    # Show first few rows
    print("\nFirst 10 rows:")
    print(submission_zero.head(10))
    
except FileNotFoundError:
    print("❌ No submission file found. Need to run main.py first.")
    submission_zero = None

✓ Submission file loaded: 28512 rows
Sales statistics:
  Min: $0.00
  Max: $16574.79
  Mean: $493.50
  Unique values: 27356

First 10 rows:
        id        sales
0  3000888     4.618296
1  3000889     0.000000
2  3000890     3.642586
3  3000891  2435.995838
4  3000892     0.348912
5  3000893   466.477860
6  3000894    16.137880
7  3000895   837.301114
8  3000896   885.084859
9  3000897   147.604287


In [14]:
# Load the original data to understand structure
print("=== LOADING DATA ===")

# Load training data
train = pd.read_csv('train.csv', 
                   usecols=['store_nbr', 'family', 'date', 'sales'],
                   dtype={'store_nbr': 'category', 'family': 'category'},
                   parse_dates=['date'])
train['date'] = train.date.dt.to_period('D')
train = train.set_index(['date', 'family', 'store_nbr']).sort_index()

print(f"Training data shape: {train.shape}")
print(f"Training period: {train.index.get_level_values('date').min()} to {train.index.get_level_values('date').max()}")

# Load test data
test_orig = pd.read_csv('test.csv')
test_orig['date'] = pd.to_datetime(test_orig['date']).dt.to_period('D')

print(f"Test data shape: {test_orig.shape}")
print(f"Test period: {test_orig['date'].min()} to {test_orig['date'].max()}")
print(f"Test columns: {test_orig.columns.tolist()}")

# Show sample of test data
print("\nTest data sample:")
print(test_orig.head())

=== LOADING DATA ===
Training data shape: (3000888, 1)
Training period: 2013-01-01 to 2017-08-15
Test data shape: (28512, 5)
Test period: 2017-08-16 to 2017-08-31
Test columns: ['id', 'date', 'store_nbr', 'family', 'onpromotion']

Test data sample:
        id        date  store_nbr      family  onpromotion
0  3000888  2017-08-16          1  AUTOMOTIVE            0
1  3000889  2017-08-16          1   BABY CARE            0
2  3000890  2017-08-16          1      BEAUTY            2
3  3000891  2017-08-16          1   BEVERAGES           20
4  3000892  2017-08-16          1       BOOKS            0
Training data shape: (3000888, 1)
Training period: 2013-01-01 to 2017-08-15
Test data shape: (28512, 5)
Test period: 2017-08-16 to 2017-08-31
Test columns: ['id', 'date', 'store_nbr', 'family', 'onpromotion']

Test data sample:
        id        date  store_nbr      family  onpromotion
0  3000888  2017-08-16          1  AUTOMOTIVE            0
1  3000889  2017-08-16          1   BABY CARE      

## 2. Analyze the Wide Format Structure

The model was trained on wide format data where each column represents a store-family combination. Let's examine this structure.

In [15]:
# Analyze the wide format structure
print("=== WIDE FORMAT ANALYSIS ===")
print(f"Wide format shape: {y_train_wide.shape}")
print(f"Columns (first 10): {y_train_wide.columns.tolist()[:10]}")
print(f"Column structure: {type(y_train_wide.columns)}")

if hasattr(y_train_wide.columns, 'nlevels'):
    print(f"\nMultiIndex levels: {y_train_wide.columns.nlevels}")
    print(f"Level 0 (families): {y_train_wide.columns.get_level_values(0).unique()}")
    print(f"Level 1 (stores): {y_train_wide.columns.get_level_values(1).unique()[:10]}")
else:
    print(f"Regular columns: {y_train_wide.columns[:10]}")

# Show a sample of the wide format data
print("\nWide format sample (last 5 dates, first 5 columns):")
print(y_train_wide.iloc[-5:, :5])
print(f"\nWide format data type: {y_train_wide.dtypes.iloc[0]}")
print(f"Wide format stats for first column:")
print(y_train_wide.iloc[:, 0].describe())

# Create mock predictions with same structure as training
print("\n=== CREATING MOCK PREDICTIONS ===")
mock_predictions = pd.DataFrame(
    np.random.uniform(10, 100, size=(16, 1782)),  # Random values between 10-100
    index=pd.period_range('2017-08-16', periods=16, freq='D'),
    columns=y_train_wide.columns
)

print(f"Mock predictions shape: {mock_predictions.shape}")
print(f"Mock predictions stats:")
print(f"  Min: {mock_predictions.min().min():.2f}")
print(f"  Max: {mock_predictions.max().max():.2f}")
print(f"  Mean: {mock_predictions.mean().mean():.2f}")
print(f"\nMock predictions sample:")
print(mock_predictions.iloc[:3, :3])

=== WIDE FORMAT ANALYSIS ===
Wide format shape: (1684, 1782)
Columns (first 10): [('AUTOMOTIVE', '1'), ('AUTOMOTIVE', '10'), ('AUTOMOTIVE', '11'), ('AUTOMOTIVE', '12'), ('AUTOMOTIVE', '13'), ('AUTOMOTIVE', '14'), ('AUTOMOTIVE', '15'), ('AUTOMOTIVE', '16'), ('AUTOMOTIVE', '17'), ('AUTOMOTIVE', '18')]
Column structure: <class 'pandas.core.indexes.multi.MultiIndex'>

MultiIndex levels: 2
Level 0 (families): CategoricalIndex(['AUTOMOTIVE', 'BABY CARE', 'BEAUTY', 'BEVERAGES', 'BOOKS',
                  'BREAD/BAKERY', 'CELEBRATION', 'CLEANING', 'DAIRY', 'DELI',
                  'EGGS', 'FROZEN FOODS', 'GROCERY I', 'GROCERY II',
                  'HARDWARE', 'HOME AND KITCHEN I', 'HOME AND KITCHEN II',
                  'HOME APPLIANCES', 'HOME CARE', 'LADIESWEAR',
                  'LAWN AND GARDEN', 'LINGERIE', 'LIQUOR,WINE,BEER',
                  'MAGAZINES', 'MEATS', 'PERSONAL CARE', 'PET SUPPLIES',
                  'PLAYERS AND ELECTRONICS', 'POULTRY', 'PREPARED FOODS',
             

## 3. Simulate Prediction Process

Let's simulate the prediction process step by step to identify where the zeros are coming from.

In [16]:
# Create mock predictions to test the pipeline
print("=== TESTING PREDICTION PIPELINE ===")

# Create fake predictions with known non-zero values
n_test_periods = 16  # From the output
n_columns = 1782     # From the output

# Create mock predictions with realistic values
np.random.seed(42)
mock_predictions = np.random.uniform(10, 100, size=(n_test_periods, n_columns))

print(f"Mock predictions shape: {mock_predictions.shape}")
print(f"Mock prediction range: {mock_predictions.min():.2f} - {mock_predictions.max():.2f}")
print(f"Mock prediction mean: {mock_predictions.mean():.2f}")

# Create test dates (same as in the model)
test_dates = pd.period_range('2017-08-16', '2017-08-31', freq='D')
print(f"Test dates: {test_dates[:5]} ... {test_dates[-2:]}")

# Create prediction DataFrame using the same column structure
pred_df = pd.DataFrame(mock_predictions, 
                      index=test_dates,
                      columns=y_train_wide.columns)

print(f"Prediction DataFrame shape: {pred_df.shape}")
print("Prediction DataFrame sample:")
print(pred_df.iloc[:3, :3])

=== TESTING PREDICTION PIPELINE ===
Mock predictions shape: (16, 1782)
Mock prediction range: 10.00 - 99.99
Mock prediction mean: 54.99
Test dates: PeriodIndex(['2017-08-16', '2017-08-17', '2017-08-18', '2017-08-19',
             '2017-08-20'],
            dtype='period[D]') ... PeriodIndex(['2017-08-30', '2017-08-31'], dtype='period[D]')
Prediction DataFrame shape: (16, 1782)
Prediction DataFrame sample:
family     AUTOMOTIVE                      
store_nbr           1         10         11
2017-08-16  43.708611  95.564288  75.879455
2017-08-17  51.532972  67.348133  69.341853
2017-08-18  51.258828  75.250547  61.685882


In [17]:
# Test the stacking process (this is where the problem likely occurs)
print("=== TESTING STACKING PROCESS ===")

# Stack the predictions back to long format
pred_stacked = pred_df.stack(['family', 'store_nbr']).to_frame()
pred_stacked.columns = ['sales']
pred_stacked = pred_stacked.reset_index()

print(f"Stacked shape: {pred_stacked.shape}")
print(f"Stacked columns: {pred_stacked.columns.tolist()}")
print("Stacked sample:")
print(pred_stacked.head(10))

# Check if we have non-zero values
print("\nStacked sales statistics:")
print(f"Min: {pred_stacked['sales'].min():.2f}")
print(f"Max: {pred_stacked['sales'].max():.2f}")
print(f"Mean: {pred_stacked['sales'].mean():.2f}")
print(f"Non-zero count: {(pred_stacked['sales'] > 0).sum()}")

# Check the column names after stacking
print("\nColumn data types:")
print(pred_stacked.dtypes)

=== TESTING STACKING PROCESS ===
Stacked shape: (28512, 4)
Stacked columns: ['level_0', 'family', 'store_nbr', 'sales']
Stacked sample:
      level_0      family store_nbr      sales
0  2017-08-16  AUTOMOTIVE         1  43.708611
1  2017-08-16  AUTOMOTIVE        10  95.564288
2  2017-08-16  AUTOMOTIVE        11  75.879455
3  2017-08-16  AUTOMOTIVE        12  63.879264
4  2017-08-16  AUTOMOTIVE        13  24.041678
5  2017-08-16  AUTOMOTIVE        14  24.039507
6  2017-08-16  AUTOMOTIVE        15  15.227525
7  2017-08-16  AUTOMOTIVE        16  87.955853
8  2017-08-16  AUTOMOTIVE        17  64.100351
9  2017-08-16  AUTOMOTIVE        18  73.726532

Stacked sales statistics:
Min: 10.00
Max: 99.99
Mean: 54.99
Non-zero count: 28512

Column data types:
level_0      period[D]
family        category
store_nbr     category
sales          float64
dtype: object


In [18]:
# Test merging with test data
print("=== TESTING MERGE PROCESS ===")

# The issue is in the column name after stacking - it should be 'date' not 'level_0'
# Let's fix the stacked data column names
pred_stacked_fixed = pred_stacked.copy()

# The first column after reset_index should be the date
if 'level_0' in pred_stacked_fixed.columns:
    pred_stacked_fixed = pred_stacked_fixed.rename(columns={'level_0': 'date'})

print(f"Fixed stacked columns: {pred_stacked_fixed.columns.tolist()}")
print("Fixed stacked sample:")
print(pred_stacked_fixed.head())

# Now test the merge
print(f"\nTest data columns: {test_orig.columns.tolist()}")
print("Test data sample:")
print(test_orig[['id', 'date', 'store_nbr', 'family']].head())

# Try merging on the correct columns
try:
    merged = test_orig.merge(
        pred_stacked_fixed[['date', 'family', 'store_nbr', 'sales']], 
        on=['date', 'family', 'store_nbr'], 
        how='left'
    )
    
    print("\nMerge successful!")
    print(f"Merged shape: {merged.shape}")
    print("Merged sales stats:")
    print(f"  Min: {merged['sales'].min():.2f}")
    print(f"  Max: {merged['sales'].max():.2f}")
    print(f"  Mean: {merged['sales'].mean():.2f}")
    print(f"  Non-null count: {merged['sales'].notna().sum()}")
    
    # Show sample of merged data
    print("\nMerged data sample:")
    print(merged[['id', 'date', 'family', 'store_nbr', 'sales']].head(10))
    
except Exception as e:
    print(f"Merge failed: {e}")
    print("Trying to debug data types...")
    print(f"Test date type: {test_orig['date'].dtype}")
    print(f"Pred date type: {pred_stacked_fixed['date'].dtype}")
    print(f"Test family type: {test_orig['family'].dtype}")
    print(f"Pred family type: {pred_stacked_fixed['family'].dtype}")

=== TESTING MERGE PROCESS ===
Fixed stacked columns: ['date', 'family', 'store_nbr', 'sales']
Fixed stacked sample:
         date      family store_nbr      sales
0  2017-08-16  AUTOMOTIVE         1  43.708611
1  2017-08-16  AUTOMOTIVE        10  95.564288
2  2017-08-16  AUTOMOTIVE        11  75.879455
3  2017-08-16  AUTOMOTIVE        12  63.879264
4  2017-08-16  AUTOMOTIVE        13  24.041678

Test data columns: ['id', 'date', 'store_nbr', 'family', 'onpromotion']
Test data sample:
        id        date  store_nbr      family
0  3000888  2017-08-16          1  AUTOMOTIVE
1  3000889  2017-08-16          1   BABY CARE
2  3000890  2017-08-16          1      BEAUTY
3  3000891  2017-08-16          1   BEVERAGES
4  3000892  2017-08-16          1       BOOKS

Merge successful!
Merged shape: (28512, 6)
Merged sales stats:
  Min: nan
  Max: nan
  Mean: nan
  Non-null count: 0

Merged data sample:
        id        date        family store_nbr  sales
0  3000888  2017-08-16    AUTOMOTIVE      

In [19]:
# Debug the merge failure
print("=== DEBUGGING MERGE FAILURE ===")

print("Data type comparison:")
print(f"Test date type: {test_orig['date'].dtype}")
print(f"Pred date type: {pred_stacked_fixed['date'].dtype}")
print(f"Test store_nbr type: {test_orig['store_nbr'].dtype}")
print(f"Pred store_nbr type: {pred_stacked_fixed['store_nbr'].dtype}")
print(f"Test family type: {test_orig['family'].dtype}")
print(f"Pred family type: {pred_stacked_fixed['family'].dtype}")

print("\nSample values comparison:")
print("Test data first 5 unique combinations:")
test_sample = test_orig[['date', 'family', 'store_nbr']].head()
print(test_sample)

print("\nPrediction data first 5 rows:")
pred_sample = pred_stacked_fixed[['date', 'family', 'store_nbr']].head()
print(pred_sample)

print("\nChecking if date formats match:")
print(f"Test date samples: {test_orig['date'].head().tolist()}")
print(f"Pred date samples: {pred_stacked_fixed['date'].head().tolist()}")

# Try to fix data types for merge
pred_stacked_for_merge = pred_stacked_fixed.copy()

# Convert Period to datetime if needed
if pred_stacked_for_merge['date'].dtype.name.startswith('period'):
    pred_stacked_for_merge['date'] = pred_stacked_for_merge['date'].dt.to_timestamp().dt.date
    pred_stacked_for_merge['date'] = pd.to_datetime(pred_stacked_for_merge['date'])

# Convert store_nbr to integer if needed  
pred_stacked_for_merge['store_nbr'] = pred_stacked_for_merge['store_nbr'].astype(str).astype(int)

print(f"\nAfter conversion:")
print(f"Pred date type: {pred_stacked_for_merge['date'].dtype}")
print(f"Pred store_nbr type: {pred_stacked_for_merge['store_nbr'].dtype}")

# Test merge again
merged_fixed = test_orig.merge(
    pred_stacked_for_merge[['date', 'family', 'store_nbr', 'sales']], 
    on=['date', 'family', 'store_nbr'], 
    how='left'
)

print(f"\nFixed merge stats:")
print(f"Merged shape: {merged_fixed.shape}")
print(f"Non-null sales: {merged_fixed['sales'].notna().sum()}")
print(f"Sales range: {merged_fixed['sales'].min():.2f} - {merged_fixed['sales'].max():.2f}")
print(f"\nFixed merged sample:")
print(merged_fixed[['id', 'date', 'family', 'store_nbr', 'sales']].head(10))

=== DEBUGGING MERGE FAILURE ===
Data type comparison:
Test date type: period[D]
Pred date type: period[D]
Test store_nbr type: int64
Pred store_nbr type: category
Test family type: object
Pred family type: category

Sample values comparison:
Test data first 5 unique combinations:
         date      family  store_nbr
0  2017-08-16  AUTOMOTIVE          1
1  2017-08-16   BABY CARE          1
2  2017-08-16      BEAUTY          1
3  2017-08-16   BEVERAGES          1
4  2017-08-16       BOOKS          1

Prediction data first 5 rows:
         date      family store_nbr
0  2017-08-16  AUTOMOTIVE         1
1  2017-08-16  AUTOMOTIVE        10
2  2017-08-16  AUTOMOTIVE        11
3  2017-08-16  AUTOMOTIVE        12
4  2017-08-16  AUTOMOTIVE        13

Checking if date formats match:
Test date samples: [Period('2017-08-16', 'D'), Period('2017-08-16', 'D'), Period('2017-08-16', 'D'), Period('2017-08-16', 'D'), Period('2017-08-16', 'D')]
Pred date samples: [Period('2017-08-16', 'D'), Period('2017-08

In [20]:
# Check key matching in detail
print("=== DETAILED KEY MATCHING ===")

# Convert test date to datetime for comparison
test_for_comparison = test_orig.copy()
test_for_comparison['date'] = test_for_comparison['date'].dt.to_timestamp().dt.date
test_for_comparison['date'] = pd.to_datetime(test_for_comparison['date'])

print("Creating merge keys:")
test_keys = test_for_comparison[['date', 'family', 'store_nbr']].drop_duplicates()
pred_keys = pred_stacked_for_merge[['date', 'family', 'store_nbr']].drop_duplicates()

print(f"Test unique keys: {len(test_keys)}")
print(f"Pred unique keys: {len(pred_keys)}")

print(f"\nFirst 10 test keys:")
print(test_keys.head(10))

print(f"\nFirst 10 pred keys:")
print(pred_keys.head(10))

# Check for exact matches
print("\nTesting specific matches:")
test_key1 = ('2017-08-16', 'AUTOMOTIVE', 1)
pred_match = pred_stacked_for_merge[
    (pred_stacked_for_merge['date'] == pd.to_datetime('2017-08-16')) &
    (pred_stacked_for_merge['family'] == 'AUTOMOTIVE') &
    (pred_stacked_for_merge['store_nbr'] == 1)
]
print(f"Looking for {test_key1}")
print(f"Found {len(pred_match)} matches in predictions:")
if len(pred_match) > 0:
    print(pred_match[['date', 'family', 'store_nbr', 'sales']])

# Check if any keys match at all
merged_keys = test_keys.merge(pred_keys, on=['date', 'family', 'store_nbr'], how='inner')
print(f"\nTotal matching keys: {len(merged_keys)}")

if len(merged_keys) == 0:
    print("NO KEYS MATCH! Let's debug further...")
    
    # Check each column individually
    test_dates = set(test_for_comparison['date'].unique())
    pred_dates = set(pred_stacked_for_merge['date'].unique())
    print(f"Date overlap: {len(test_dates & pred_dates)} of {len(test_dates)} test dates")
    print(f"Test dates: {sorted(list(test_dates))[:5]}")
    print(f"Pred dates: {sorted(list(pred_dates))[:5]}")
    
    test_families = set(test_for_comparison['family'].unique())
    pred_families = set(pred_stacked_for_merge['family'].unique())
    print(f"Family overlap: {len(test_families & pred_families)} of {len(test_families)} test families")
    
    test_stores = set(test_for_comparison['store_nbr'].unique())
    pred_stores = set(pred_stacked_for_merge['store_nbr'].unique())
    print(f"Store overlap: {len(test_stores & pred_stores)} of {len(test_stores)} test stores")
    
else:
    print(f"Matching keys sample:")
    print(merged_keys.head())

=== DETAILED KEY MATCHING ===
Creating merge keys:
Test unique keys: 28512
Pred unique keys: 28512

First 10 test keys:
        date        family  store_nbr
0 2017-08-16    AUTOMOTIVE          1
1 2017-08-16     BABY CARE          1
2 2017-08-16        BEAUTY          1
3 2017-08-16     BEVERAGES          1
4 2017-08-16         BOOKS          1
5 2017-08-16  BREAD/BAKERY          1
6 2017-08-16   CELEBRATION          1
7 2017-08-16      CLEANING          1
8 2017-08-16         DAIRY          1
9 2017-08-16          DELI          1

First 10 pred keys:
        date      family  store_nbr
0 2017-08-16  AUTOMOTIVE          1
1 2017-08-16  AUTOMOTIVE         10
2 2017-08-16  AUTOMOTIVE         11
3 2017-08-16  AUTOMOTIVE         12
4 2017-08-16  AUTOMOTIVE         13
5 2017-08-16  AUTOMOTIVE         14
6 2017-08-16  AUTOMOTIVE         15
7 2017-08-16  AUTOMOTIVE         16
8 2017-08-16  AUTOMOTIVE         17
9 2017-08-16  AUTOMOTIVE         18

Testing specific matches:
Looking for ('2017